In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по лошадям.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Лошади
1648,ОБЛАСТЬ АБАЙ,2023-03-01,2288.39
327,АЛМАТИНСКАЯ ОБЛАСТЬ,2022-04-01,1230.38
475,АТЫРАУСКАЯ ОБЛАСТЬ,2024-08-01,683.30
94,АКМОЛИНСКАЯ ОБЛАСТЬ,2022-11-01,1948.10
12,АКМОЛИНСКАЯ ОБЛАСТЬ,2016-01-01,1341.67
1767,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2017-12-01,2135.83
1952,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2023-05-01,476.70
812,ГАСТАНА,2022-09-01,1.62
1635,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-09-01,574.90
577,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2023-02-01,825.75


In [3]:
regions = df['Регион'].unique()
target   = "Лошади"
horizon  = 3
epsilon = 1e-6

In [4]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [5]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[1591.57, 1062.7, 1010.87]","[1655.9, 915.95, 1074.61]",99.56,91.61,8.61
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[1067.12, 1015.0, 897.83]","[915.95, 1074.61, 852.53]",97.40,85.36,9.12
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[1000.35, 883.41, 1162.72]","[1074.61, 852.53, 1126.55]",50.91,47.10,4.58
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[889.38, 1169.08, 1901.7]","[852.53, 1126.55, 2123.03]",131.85,100.24,6.17
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[1165.64, 1897.49, 1059.77]","[1126.55, 2123.03, 1113.91]",135.80,106.26,6.32
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[2602.05, 2359.82, 2036.06]","[2246.84, 2091.29, 1955.9]",261.22,234.63,10.92
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[2147.41, 1855.44, 2352.38]","[2091.29, 1955.9, 3893.88]",892.46,566.03,15.80
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[1823.99, 2313.67, 3097.32]","[1955.9, 3893.88, 4357.98]",1169.58,990.93,25.42
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[2417.39, 3234.31, 4083.19]","[3893.88, 4357.98, 5998.64]",1539.65,1505.20,31.88


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,93.92,76.80,5.74
1,АКТЮБИНСКАЯ ОБЛАСТЬ,213.23,177.46,9.11
2,АЛМАТИНСКАЯ ОБЛАСТЬ,180.47,129.18,8.80
3,АТЫРАУСКАЯ ОБЛАСТЬ,71.63,57.12,6.41
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,393.44,329.91,23.31
5,ГАЛМАТЫ,0.50,0.39,NaN
6,ГАСТАНА,0.82,0.75,38.23
7,ГШЫМКЕНТ,125.79,84.07,54.85
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,95.91,75.49,4.50
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,115.08,91.53,7.54


## SARIMA

In [6]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(0, 0, 0)","(1, 1, 0, 12)",97.11,76.08,7.44,"[1609.65, 1076.2, 1052.86]","[1655.9, 915.95, 1074.61]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(0, 0, 0)","(1, 1, 0, 12)",98.62,78.67,8.62,"[1076.51, 1053.53, 906.89]","[915.95, 1074.61, 852.53]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(0, 0, 0)","(1, 1, 0, 12)",50.37,46.89,4.69,"[1052.5, 904.96, 1192.69]","[1074.61, 852.53, 1126.55]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(0, 0, 0)","(1, 1, 0, 12)",116.45,100.57,6.88,"[904.27, 1193.4, 1939.9]","[852.53, 1126.55, 2123.03]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(0, 0, 0)","(1, 1, 0, 12)",113.06,92.18,5.66,"[1193.21, 1941.01, 1086.06]","[1126.55, 2123.03, 1113.91]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(1, 0, 0)","(2, 1, 0, 12)",661.97,660.05,31.57,"[2937.42, 2680.26, 2656.5]","[2246.84, 2091.29, 1955.9]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(1, 0, 0)","(2, 1, 0, 12)",332.62,295.30,13.62,"[2329.66, 2460.55, 3751.02]","[2091.29, 1955.9, 3893.88]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(1, 0, 0)","(2, 1, 0, 12)",262.34,229.00,8.82,"[2308.59, 3612.29, 4410.68]","[1955.9, 3893.88, 4357.98]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(1, 0, 0)","(0, 1, 1, 12)",522.64,442.64,10.03,"[3094.55, 4236.54, 5591.5]","[3893.88, 4357.98, 5998.64]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,83.46,67.65,5.15
1,АКТЮБИНСКАЯ ОБЛАСТЬ,174.64,146.34,7.90
2,АЛМАТИНСКАЯ ОБЛАСТЬ,168.18,138.81,9.07
3,АТЫРАУСКАЯ ОБЛАСТЬ,61.28,53.94,6.60
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,327.90,274.48,18.16
5,ГАЛМАТЫ,0.42,0.28,3866103693216970240.00
6,ГАСТАНА,0.76,0.68,31.94
7,ГШЫМКЕНТ,129.18,89.40,76.21
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,91.75,76.42,5.01
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,75.54,61.71,5.51


## Facebook Prophet

In [7]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


18:32:14 - cmdstanpy - INFO - Chain [1] start processing
18:32:14 - cmdstanpy - INFO - Chain [1] done processing
18:32:15 - cmdstanpy - INFO - Chain [1] start processing
18:32:15 - cmdstanpy - INFO - Chain [1] done processing
18:32:15 - cmdstanpy - INFO - Chain [1] start processing
18:32:15 - cmdstanpy - INFO - Chain [1] done processing
18:32:15 - cmdstanpy - INFO - Chain [1] start processing
18:32:15 - cmdstanpy - INFO - Chain [1] done processing
18:32:16 - cmdstanpy - INFO - Chain [1] start processing
18:32:16 - cmdstanpy - INFO - Chain [1] done processing
18:32:16 - cmdstanpy - INFO - Chain [1] start processing
18:32:16 - cmdstanpy - INFO - Chain [1] done processing
18:32:16 - cmdstanpy - INFO - Chain [1] start processing
18:32:17 - cmdstanpy - INFO - Chain [1] done processing
18:32:17 - cmdstanpy - INFO - Chain [1] start processing
18:32:17 - cmdstanpy - INFO - Chain [1] done processing
18:32:17 - cmdstanpy - INFO - Chain [1] start processing
18:32:17 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,123.15,108.33,9.01,"[1540.74, 949.35, 898.19]","[1655.9, 915.95, 1074.61]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,99.32,71.51,6.93,"[954.59, 907.19, 861.02]","[915.95, 1074.61, 852.53]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,105.31,78.62,7.27,"[902.04, 856.88, 1185.48]","[1074.61, 852.53, 1126.55]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,64.88,57.51,4.14,"[867.61, 1203.87, 2042.89]","[852.53, 1126.55, 2123.03]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,69.43,67.12,4.81,"[1201.73, 2039.34, 1071.42]","[1126.55, 2123.03, 1113.91]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,596.71,527.12,24.73,"[2832.58, 2927.82, 2114.99]","[2246.84, 2091.29, 1955.9]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,776.53,649.18,22.89,"[2834.78, 2042.47, 2776.42]","[2091.29, 1955.9, 3893.88]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,962.24,792.83,19.52,"[1978.81, 2678.68, 3217.59]","[1955.9, 3893.88, 4357.98]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,1470.89,1450.02,30.70,"[2589.53, 3109.94, 4200.97]","[3893.88, 4357.98, 5998.64]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,72.89,58.63,4.46
1,АКТЮБИНСКАЯ ОБЛАСТЬ,247.19,209.54,10.82
2,АЛМАТИНСКАЯ ОБЛАСТЬ,554.20,470.49,29.28
3,АТЫРАУСКАЯ ОБЛАСТЬ,136.08,119.09,13.44
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,594.23,566.00,34.60
5,ГАЛМАТЫ,0.42,0.28,NaN
6,ГАСТАНА,1.07,0.91,42.89
7,ГШЫМКЕНТ,112.04,71.90,43.07
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,164.26,136.28,8.13
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,168.19,137.52,12.78


In [8]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW            MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     5.74                   5.15          4.46     Prophet
           АКТЮБИНСКАЯ ОБЛАСТЬ     9.11                   7.90         10.82      SARIMA
           АЛМАТИНСКАЯ ОБЛАСТЬ     8.80                   9.07         29.28          HW
            АТЫРАУСКАЯ ОБЛАСТЬ     6.41                   6.60         13.44          HW
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    23.31                  18.16         34.60      SARIMA
                       ГАЛМАТЫ      NaN 3866103693216970240.00           NaN      SARIMA
                       ГАСТАНА    38.23                  31.94         42.89      SARIMA
                      ГШЫМКЕНТ    54.85                  76.21         43.07     Prophet
            ЖАМБЫЛСКАЯ ОБЛАСТЬ     4.50                   5.01          8.13          HW
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     7.54                   5.51         12.78      SARIMA
        КАРАГАНДИНСКА